In [1]:
!pip install ultralytics

     -------------------------------------- 756.5/756.5 kB 3.2 MB/s eta 0:00:00


In [2]:

#將.xml轉成Yolo所需 .txt檔案
import os
import xml.etree.ElementTree as ET

def convert_coordinates(size, box):
    # 轉換坐標
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders):
    # 解析XML並轉換為YOLO格式
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for output_folder in output_folders:
        folder_name = os.path.basename(output_folder)
        output_dir = os.path.join(output_root, folder_name)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for obj in root.findall('object'):
                cls = obj.find('name').text
                if cls not in class_dict:
                    continue
                cls_id = class_dict[cls]
                xml_box = obj.find('bndbox')
                box = (float(xml_box.find('xmin').text), float(xml_box.find('xmax').text), 
                       float(xml_box.find('ymin').text), float(xml_box.find('ymax').text))
                bb = convert_coordinates((w,h), box)
                f.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

# 資料夾列表
folders = ["訓練集_xml", "驗證集_xml", "測試集_xml"]
o_folders = ["train", "val", "test"]

# 分類字典，將類別名映射到整數標籤
class_dict = {"container": 0}

# 輸出根資料夾路徑
output_root = "E:\Downloads\貨櫃資料集\labels"

# 如果輸出根資料夾不存在，則創建
if not os.path.exists(output_root):
    os.makedirs(output_root)

# 迴圈處理每個資料夾
for folder, o_folder in zip(folders, o_folders):
    folder_path = os.path.join("E:\Downloads\貨櫃資料集", folder)  # 資料夾路徑
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]  # 獲取所有XML檔案
    output_folders = [os.path.join(output_root, o_folder)]  # 輸出資料夾路徑
    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)  # XML檔案路徑
        convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders)
        
print("labels轉換完成!")

In [6]:
from PIL import Image

# 資料夾列表
folder_path = "E:/Downloads/貨櫃資料集/"
folders = [folder_path + "訓練集", folder_path + "驗證集", folder_path + "測試集"]

output_folders = [folder_path + "images/train", folder_path + "images/val", folder_path + "images/test"]


# 創建輸出資料夾
for output_folder in output_folders:
    os.makedirs(output_folder, exist_ok=True)

# 轉換函數
def resize_images(folder, output_folder):
    # 獲取資料夾中所有圖片的檔案名稱
    files = os.listdir(folder)
    
    # 迴圈處理每張圖片
    for file in files:
        # 檔案路徑
        file_path = os.path.join(folder, file)
        
        # 如果是檔案
        if os.path.isfile(file_path):
            # 打開圖片
            img = Image.open(file_path)
            
            # 重新調整大小
            resized_img = img.resize((416, 416))
            
            # 另存新圖片
            resized_img.save(os.path.join(output_folder, file))
                
# 對每個資料夾和對應的輸出資料夾調用resize_images函數
for folder, output_folder in zip(folders, output_folders):
    resize_images(folder, output_folder)
    
print("images轉換完成!")

images轉換完成!


In [ ]:
import time
from ultralytics import YOLO

if __name__=='__main__':
    model = YOLO("yolov8n.pt")
    print("開始訓練 .........")
    t1=time.time()
    model.train(data="E:/Downloads/貨櫃資料集/data.yaml", epochs=100, imgsz=640)
    t2=time.time()
    print(f'訓練花費時間 : {t2-t1}秒')#2131.11984705925秒, epochs : 121
    path=model.export()
    print(f'模型匯出路徑 : {path}')

開始訓練 .........
Ultralytics YOLOv8.2.11  Python-3.9.13 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=E:/Downloads//data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

train: Scanning E:\Downloads\貨櫃資料集\labels\train... 2125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2125/21


train: New cache created: E:\Downloads\\labels\train.cache
